In [249]:
%reset -f
import pandas as pd
import requests
import json

In [250]:
# api endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_history_url = 'https://fantasy.premierleague.com/api/element-summary/{player_id}/'

In [251]:
# making request
response = requests.get(url)

# check status
if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

The API request was successful.


In [252]:
# get all players
response = requests.get(url)
players = response.json()['elements']

# print(json.dumps(players, indent=4, sort_keys=True))

In [253]:
# mapping positions
element_types = response.json()['element_types']
position_mapping = {}

for pos in element_types:
    position_mapping[pos['id']] = pos['singular_name']

In [254]:
# extract player info
player_info = []

for player in players:
    player_id = player['id']
    name = player['first_name'] + ' ' + player['second_name']
    position = position_mapping[player['element_type']]
    points = player['total_points']
    bonus = player['bonus']
    value = player['now_cost']/10
    minutes = player['minutes']
    ppg = player['points_per_game']  
    goals = player['goals_scored']
    xg = player['expected_goals']
    xg90 = player['expected_goals_per_90']
    assists = player['assists']
    xa = player['expected_assists']
    xa90 = player['expected_assists_per_90']
    cs = player['clean_sheets']
    goals_conc = player['goals_conceded']
    saves = player['saves']
    influence = player['influence']
    threat = player['threat']  
    creativity = player['creativity']
    transfers_in = player['transfers_in_event']
    transfers_out = player['transfers_out_event']
  
    player_info.append({
        'name': name,
        'pos': position[:1],
        'value': value,
        'mins': minutes,
        'pts': points,
        'pts-bns': int(points) - bonus,
        'ppg': ppg,
        'goals': goals,
        'xg': float(xg),
        'xg90': xg90,
        'ass': assists,
        'xa': float(xa),
        'xa90': xa90,
        'cs': cs,
        'g_conc': goals_conc,
        'saves': saves,
        'influence': influence,
        'threat': threat,
        'creativity': creativity,
        'trans_in': transfers_in,
        'trans_out': transfers_out
    })
    
df = pd.DataFrame(player_info)

In [255]:
# midfielders
mid = df.drop(columns=['cs', 'g_conc','saves'])
mid.insert(6, 'xp', mid.assign(xp = mid['xa'] * 3 + mid['xg'] * 5)['xp'])
filtered_mid = mid[(mid['pos'] == 'M') & ((mid['xg90'] > 0.2) | (mid['xa90'] > 0.2)) & (mid['mins'] > 90) ]
sorted_mid = filtered_mid.sort_values(by='xp', ascending=False)

sorted_mid

,name,pos,value,mins,pts,pts-bns,xp,ppg,goals,xg,xg90,ass,xa,xa90,influence,threat,creativity,trans_in,trans_out
118,Bryan Mbeumo,M,6.8,360,33,29,22.86,8.2,4,4.02,1.00,0,0.92,0.23,164.0,159.0,108.6,83427,31838
17,Bukayo Saka,M,8.7,358,25,21,15.60,6.2,2,2.16,0.54,1,1.60,0.40,150.4,166.0,207.9,28517,34660
453,Bruno Borges Fernandes,M,8.5,360,18,15,15.28,4.5,1,2.00,0.50,1,1.76,0.44,129.0,85.0,188.6,20999,39070
368,Mohamed Salah,M,12.5,341,23,23,14.28,5.8,2,2.49,0.66,2,0.61,0.16,131.8,222.0,101.2,19451,38612
620,James Maddison,M,7.8,337,29,25,13.40,7.2,2,1.60,0.43,2,1.80,0.48,162.2,171.0,168.4,203551,9962
646,Jarrod Bowen,M,7.1,360,30,25,13.40,7.5,3,2.38,0.60,1,0.50,0.12,147.0,177.0,61.6,64533,26948
264,Eberechi Eze,M,6.3,360,15,14,11.49,3.8,1,1.74,0.44,0,0.93,0.23,101.6,128.0,192.7,6383,33168
226,Enzo Fernández,M,5.0,360,5,5,10.64,1.2,0,1.36,0.34,0,1.28,0.32,65.6,61.0,193.3,5656,17986
291,Abdoulaye Doucouré,M,5.5,360,14,12,10.43,3.5,1,1.93,0.48,0,0.26,0.06,75.6,165.0,86.1,876,630
690,Pedro Lomba Neto,M,5.5,360,20,17,9.66,5.0,0,1.11,0.28,3,1.37,0.34,109.4,106.0,202.3,1512,670


In [256]:
# forwards
forw = df.drop(columns=['cs', 'g_conc','saves'])
forw.insert(6, 'xp', forw.assign(xp = forw['xa'] * 3 + forw['xg'] * 4)['xp'])
filtered_forw = forw[(forw['pos'] == 'F') & ((forw['xg90'] > 0.2) | (forw['xa90'] > 0.2)) & (forw['mins'] > 90) ]
sorted_forw = filtered_forw.sort_values(by='xp', ascending=False)

sorted_forw

,name,pos,value,mins,pts,pts-bns,xp,ppg,goals,xg,xg90,ass,xa,xa90,influence,threat,creativity,trans_in,trans_out
433,Erling Haaland,F,14.1,349,39,33,18.70,9.8,6,4.54,1.17,1,0.18,0.05,224.4,248.0,71.8,37342,3048
235,Nicolas Jackson,F,7.2,352,10,9,13.72,2.5,1,2.95,0.75,0,0.64,0.16,69.2,187.0,61.8,11811,145713
263,Odsonne Edouard,F,5.5,339,24,20,10.68,6.0,3,2.49,0.66,0,0.24,0.06,120.6,241.0,39.2,12258,6473
144,Evan Ferguson,F,5.9,205,25,22,10.57,6.2,4,2.56,1.12,0,0.11,0.05,147.8,163.0,18.7,184436,10652
128,Yoane Wissa,F,6.2,329,24,19,10.08,6.0,2,2.46,0.67,1,0.08,0.02,84.2,152.0,27.0,22576,97541
422,Julián Álvarez,F,6.6,359,28,24,9.53,7.0,2,1.94,0.49,3,0.59,0.15,115.8,137.0,170.2,205732,11172
698,Matheus Santos Carneiro Da Cunha,F,5.5,340,12,12,8.82,3.0,1,1.92,0.51,0,0.38,0.10,66.6,103.0,57.5,1355,2669
534,Taiwo Awoniyi,F,6.6,246,25,22,8.19,6.2,3,1.83,0.67,1,0.29,0.11,122.4,100.0,48.3,66919,10106
89,Dominic Solanke,F,6.5,360,21,19,7.67,5.2,2,1.76,0.44,1,0.21,0.05,103.2,142.0,67.6,4742,5346
147,João Pedro Junqueira de Jesus,F,5.5,226,10,9,7.40,2.5,1,1.73,0.69,0,0.16,0.06,60.0,114.0,59.6,6559,102622
